#[Your_name]

<h1 style="text-align: center;">Text Classification<h1>

Prepare libraries

In [ ]:
# here put every import you need e.g. import nltk
# it's better to load what you need from the package by from [] import [] instead of import the whole package
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import Sequential
from keras import layers
from keras.layers import Dense, Conv1D, AveragePooling1D, Flatten
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
import re
import numpy
from cleantext import clean
from arabicstopwords.stopwords_lexicon import stopwords_lexicon
import nltk
from wordcloud import WordCloud
from arabic_reshaper import arabic_reshaper
from bidi.algorithm import get_display
from nltk import bigrams, trigrams
from collections import Counter
from snowballstemmer import stemmer
from tashaphyne.stemming import ArabicLightStemmer
from farasa.segmenter import FarasaSegmenter
from farasa.stemmer import FarasaStemmer
from matplotlib.font_manager import FontProperties

download 

Download data

In [ ]:
# !wget 'https://drive.google.com/uc?export=download&id=1cMSjxa3nA706LIZDEhwMpaVRMY2IX9P0' -O 'data.zip'

In [ ]:
# !pip install Arabic-Stopwords
# 2.2 nltk.download('punkt') # open vpn
# 2.3 !pip install arabic-reshaper
# 2.4 nltk.download('stopwords')
# 3.8 !pip install snowballstemmer
# 3.8 !pip install Tashaphyne
# 3.8 !pip install -U farasapy

In [ ]:
# !unzip data.zip

<h2 dir="rtl">مثال عن كيفية تنظيف حلول الطلبات:</h2>


<div dir="rtl">شرح ما يقوم به الكود (like code documentation)<div>

In [ ]:
# your code here


In [ ]:
# example test

<div dir="rtl">ملاحظاتك في حال وجودها</div>
<div dir="rtl">يمكنك إضافة خلايا لكل طلب بقدر ما تشاء، المهم أن تحافظ على تنظيم الملف</div>

# Question [1]: Load the data

## [1.1]

In [ ]:
train_file_path = 'train.csv'
test_file_path = 'test.csv'
validation_file_path = 'val.csv'
train_data = pd.read_csv(train_file_path)
test_data = pd.read_csv(test_file_path)
validation_data = pd.read_csv(validation_file_path)

print("number of data in train data : ",len(train_data))
print("number of data in test data : ",len(test_data))
print("number of data in validation data : ",len(validation_data))

In [ ]:
train_data.head(10)

In [ ]:
test_data.head(10)

In [ ]:
validation_data.head(10)

In [ ]:
#drop_duplicates يزيل الصفوف المتكرره
#dropna يحذف الصفوف المكررة
test_data=test_data.drop_duplicates().dropna()
train_data=train_data.drop_duplicates().dropna()
validation_data=validation_data.drop_duplicates().dropna()
print("number of data in train data after cleaning : ",len(train_data))
print("number of data in test data after cleaning : ",len(test_data))
print("number of data in validation data after cleaning : ",len(validation_data))

In [ ]:
train_data.head(10)

In [ ]:
test_data.head(10)

In [ ]:
validation_data.head(10)

## [1.2]

In [ ]:
def unify_specialty(name):
    name = re.sub(r"[^\w\s]", " ", name)  # يزيل الرموز الترقيمية
    name = re.sub(r"\s+", " ", name).strip()  # يزيل الفراغات المتكرره #strip يزيل الفراغات يلي بالبداية والنهاية
    name =re.sub("[\_\-]"," ",name)
    return name

train_data['label']=train_data['label'].apply(unify_specialty)
test_data['label']=test_data['label'].apply(unify_specialty)
validation_data['label']=validation_data['label'].apply(unify_specialty)   


In [ ]:
x=set(train_data['label'])
x

In [ ]:
Dictionary_of_diseases= {
    'أمراض الجهاز التنفسي': 'أمراض الجهاز التنفسي',
    'الجهاز التنفسي': 'أمراض الجهاز التنفسي',
    'امراض الجهاز التنفسي': 'أمراض الجهاز التنفسي',
    'أمراض الدم': 'أمراض الدم',
    'الدم': 'أمراض الدم',
    'امراض الدم': 'أمراض الدم',
    'أمراض الغدد الصماء': 'أمراض الغدد الصماء',
    'الغدد الصماء': 'أمراض الغدد الصماء',
    'امراض الغدد الصماء': 'أمراض الغدد الصماء',
    'مرض السكري': 'مرض السكري',
    'السكري': 'مرض السكري',
    'الاورام الخبيثة والحميدة':'الأورام الخبيثة والحميدة'

}

def Unification_name_of_diseases(c):
    c=c.map(Dictionary_of_diseases).fillna(c)
    return c

In [ ]:
train_data['label']=Unification_name_of_diseases(train_data['label'])
test_data['label']=Unification_name_of_diseases(test_data['label'])
validation_data['label']=Unification_name_of_diseases(validation_data['label'])

In [ ]:
x=set(train_data['label'])
x

# Question [2]: Text Analysis

## [2.1] 


In [ ]:
train_data

In [ ]:
# Train data

# عدد الأجوبة الخاصة بأمراض الدم :
answers_len_in_blood_diseases = len(train_data[train_data['label']=='أمراض الدم']['answer'].tolist())
# print(answers_len_in_blood_diseases)

# عدد الأسئلة الخاصة بأمراض الدم :
questions_len_in_blood_diseases = len(train_data[train_data['label']=='أمراض الدم']['question'].tolist())
# print(questions_len_in_blood_diseases)


# عدد الداتا الخاصة بأمراض الدم :
total_blood_diseases_len = len(train_data[train_data['label']=='أمراض الدم'])
print('عدد الداتا الخاصة بأمراض الدم:',total_blood_diseases_len)

# عدد الداتا في أمراض الجهاز التنفسي :
total_blood_diseases_len = len(train_data[train_data['label']=='أمراض الجهاز التنفسي'])
print('عدد الداتا الخاصة بأمراض الجهاز التنفسي:',total_blood_diseases_len)

# غدد الداتا في أمراض الغدد الصماء :
total_blood_diseases_len = len(train_data[train_data['label']=='أمراض الغدد الصماء'])
print('عدد الداتا الخاصة بأمراض الغدد الصماء:',total_blood_diseases_len)

# عدد  الداتا ارتفاع ضغط الدم :
total_blood_diseases_len = len(train_data[train_data['label']=='ارتفاع ضغط الدم'])
print('عدد الداتا الخاصة بأمراض ارتفاع ضغط الدم:',total_blood_diseases_len)

# عدد الداتا في الأورام الخبيثة والحميدة :
total_blood_diseases_len = len(train_data[train_data['label']=='الأورام الخبيثة والحميدة'])
print('عدد الداتا الخاصة بالأورام الخبيثة والحميدة:',total_blood_diseases_len)

# عدد الداتا في جراحة العظام
total_blood_diseases_len = len(train_data[train_data['label']=='جراحة العظام'])
print('عدد الداتا الخاصة بجراحةالعظام:',total_blood_diseases_len)

# عدد الداتا في الجراحة العامة
total_blood_diseases_len = len(train_data[train_data['label']=='جراحة عامة'])
print('عدد الداتا الخاصة بالجراحة العامة:',total_blood_diseases_len)

# عدد الداتا في مرض السكري
total_blood_diseases_len = len(train_data[train_data['label']=='مرض السكري'])
print('عدد الداتا الخاصة بمرض السكري:',total_blood_diseases_len)


In [ ]:
# Drawing pie chart:
# ملاحظة بالعربي طلع الخط مفشكل ف تمت الكتابة بالانكليزي



data = {
    'الاختصاص': ['أمراض الدم', 'أمراض الجهاز التنفسي', 'أمراض الغدد الصماء','أمراض ارتفاع ضغط الدم','الأمراض الخبيثة والحميدة','جراحةالعظام','الجراحة العامة','مرض السكري'],
    'عدد النصوص': [2153, 6002, 9417,4190,10711,2641,10548,7096]
}

data


df = pd.DataFrame(data)

# print(df)
arabic_font = FontProperties(fname='font/NotoNaskhArabic-VariableFont_wght.ttf') 

df['الاختصاص'] = [get_display(arabic_reshaper.reshape(label)) for label in df['الاختصاص']]
plt.figure(figsize=(8, 8))
plt.pie(df['عدد النصوص'], labels=df['الاختصاص'], autopct='%1.1f%%', startangle=140)
plt.title(get_display(arabic_reshaper.reshape('توزيع النصوص حسب الاختصاص')), fontproperties=arabic_font)
plt.show()

# import pandas as pd
# import matplotlib.pyplot as plt

# data = {
#     'Specialization': ['Blood Diseases', 'Respiratory System Diseases', 'Endocrine Diseases', 'Hypertension', 'Malignant and Benign Diseases', 'Orthopedic Surgery', 'General Surgery', 'Diabetes'],
#     'Number of Texts': [1398, 3676, 5752, 2537, 6449, 2211, 6307, 4268]
# }

# df = pd.DataFrame(data)

# print(df)

# plt.figure(figsize=(8, 8))
# plt.pie(df['Number of Texts'], labels=df['Specialization'], autopct='%1.1f%%', startangle=140)
# plt.title('Distribution of Texts by Specialization')
# plt.show()


## [2.2] 

In [ ]:
def delete_repeated_letter(text):
    # cleaned_text = re.sub(r'(.)\1{2,}', r'\1', text)
    cleaned_text = re.sub(r'([^\w\s\.])\1+', r'\1', text)
    cleaned_text = re.sub(r'(\.\s*)\1+$', r'\1', cleaned_text)
    return cleaned_text

def handle_connected_words(tokenized):
    # Handle connected words with '-'
    result_tokens = []
    for token in tokenized:
        # Split connected words with '-'
        result_tokens.extend(token.split('-'))
    return result_tokens


def filter_text(list):
    list_of_phrases =[]
    all_tokens = []
    for sentence in list:
        tokenizer = RegexpTokenizer(r'\b\d+\b|\b[^\d\W_]{2,}\b|[^\d\W_]+(?:-[^\d\W_]+)?|\S')
        tokenized = tokenizer.tokenize(delete_repeated_letter(sentence))
        tokenized = handle_connected_words(tokenized)

        # Remove '-' and '/' and one-letter words from each token
        # cleaned_tokens = [token for token in tokenized if len(token) > 1 and token not in ['-', '/']]
        # cleaned_tokens = [token if len(token) > 1 or token == 'و' else '' for token in tokenized if token not in ['-', '/']]

        cleaned_tokens = [token if (token.isalpha() and len(token) > 1) or token == 'و' else '' for token in tokenized if token not in ['-', '/']]

        combined_phrase = ' '.join(cleaned_tokens)
        list_of_phrases.append(combined_phrase)

    for phrase in list_of_phrases:
        tokens = nltk.word_tokenize(phrase)
        all_tokens.append(tokens)
        
    return all_tokens



In [ ]:
train_data_answers_list = train_data['answer'].values
answers = filter_text(train_data_answers_list)
flattened_answers = [item for sublist in answers for item in sublist]
print(len(flattened_answers))


In [ ]:
train_data_questions_list = train_data['question'].values
questions = filter_text(train_data_questions_list)
flattened_quesions = [item for sublist in questions for item in sublist]
print(len(flattened_quesions))

In [ ]:
# حساب عدد الكلمات الكلي :
total_words_len = len(flattened_quesions) + len(flattened_answers)
print(total_words_len)

In [ ]:
# حساب عدد الكلمات الفريدة :
total_unique_words=len(set(flattened_quesions+flattened_answers))
print("عدد الكلمات الفريدة:", total_unique_words)

## [2.3] 

In [ ]:
WCD=dict()
train_data_answers_list = flattened_answers
train_data_questions_list = flattened_quesions
train_data_list=train_data_answers_list+train_data_questions_list
answers_text = ' '.join(train_data_list)
reshaped_text = arabic_reshaper.reshape(answers_text)
arabic_text = get_display(reshaped_text)
wordcloud = WordCloud(font_path='font/NotoNaskhArabic-VariableFont_wght.ttf', background_color='white').generate(arabic_text)
WCD=wordcloud.words_
plt.figure(figsize=(50, 50))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

## [2.4] 

In [ ]:
def tokenized_text(text):
    # allWords = nltk.tokenize.word_tokenize(text)

    allWordDist = nltk.FreqDist(w for w in text)
    stopwords = nltk.corpus.stopwords.words('english')
    allWordExceptStopDist = nltk.FreqDist(w.lower() for w in text if w not in stopwords)  
    mostCommon= allWordDist.most_common(15)
    print('أكثر 15 كلمة مكررة:',mostCommon)
    leastCommon = allWordDist.most_common()[:-11:-1]
    print('أقل 10 كلمة مكررة:', leastCommon)
    bigram_freq = nltk.FreqDist(bigrams(text))
    most_common_bigrams = bigram_freq.most_common(10)
    print('أكثر 10 ثنائيات مكررة في النص:',most_common_bigrams)
    trigram_freq = nltk.FreqDist(trigrams(text))
    most_common_trigrams = trigram_freq.most_common(10)
    print('أكثر 10 ثلاثيات كلمات مكررة في النص:',most_common_trigrams)
    collocations_bigram = nltk.collocations.BigramCollocationFinder.from_words(text)
    most_common_collocations_bigram = collocations_bigram.nbest(nltk.collocations.BigramAssocMeasures.likelihood_ratio, 10)
    print('أكثر 10 ثنائيات كلمات مهمة في النص:',most_common_collocations_bigram)
    collocations_trigram = nltk.collocations.TrigramCollocationFinder.from_words(text)
    most_common_collocations_trigram = collocations_trigram.nbest(nltk.collocations.TrigramAssocMeasures.likelihood_ratio, 10)
    print('أكثر 10 ثلاثيات كلمات مهمة في النص:',most_common_collocations_trigram)  
    print('========================================================================================================================================================================================')  


    


# tokenized_text('tony botros is tony the tony best best best tony botros in in in the whole world tony botros')
# tokenized_text(['طوني','طوني','بطرس','هو','طوني','هو','بطرس','ال','الأفضل'])
# tokenized_text(['لدي', 'جرح', 'فوق', 'حاجبي', 'و', 'تمت', 'عملية', 'الخياطة', 'ثم', 'نزع', 'الغرزات', 'بشكل', 'عادي', 'لكن', 'مر', 'اسبوع', 'على', 'ذلك', 'و', 'مازال', 'هناك', 'الم', 'و', 'تطلب', 'شديد', 'في', 'المنطقة', 'لدرجة', 'اني', 'لا', 'استطيع', 'رفع', 'حاجبي', 'ابدا', 'إضافة', 'لوجود', 'انتفاخ', 'هل', 'هدا', 'طبيعي', 'ام', 'يمكن', 'ان', 'يكون', 'خلل', 'و', 'إن', 'كان', 'كذلك', 'ما', 'هو'])
tokenized_text(answers[0])

In [ ]:
#الشرح

In [ ]:
# هاد التابع لمرق كل اختصاص وجيب منو الكلمات كلا متل مو مطلوب ب 2.4
def all_tokinized_text(specialized):
    filtered_questions = filter_text(train_data[train_data['label']==specialized]['question'].values)
    filtered_answers = filter_text(train_data[train_data['label']==specialized]['answer'].values)
    list = filtered_questions + filtered_answers
    flattened_list = [item for sublist in list for item in sublist]
    # flattened_list
    tokenized_text(flattened_list)
    

In [ ]:
all_tokinized_text('أمراض الدم')

In [ ]:
all_tokinized_text('أمراض الجهاز التنفسي')

In [ ]:
all_tokinized_text('أمراض الغدد الصماء')

In [ ]:
all_tokinized_text('ارتفاع ضغط الدم')

In [ ]:
all_tokinized_text('الأورام الخبيثة والحميدة')

In [ ]:
all_tokinized_text('جراحة العظام')

In [ ]:
all_tokinized_text('جراحة عامة')

In [ ]:
all_tokinized_text('مرض السكري')

In [ ]:
# على كل الداتا تبع الترين:
filtered_questions = filter_text(train_data['question'].values)
filtered_answers = filter_text(train_data['answer'].values)
list = filtered_questions + filtered_answers
flattened_list = [item for sublist in list for item in sublist]
# flattened_list
tokenized_text(flattened_list)


In [ ]:
#الشرح

## [2.5] 

In [ ]:
train_data['all_word']=train_data['answer']+train_data['question']
train_data_list=train_data['all_word'].values
all_word = filter_text(train_data_list)

In [ ]:
flattened_all_word = [item for sublist in all_word for item in sublist]
train_data['length_of_sentinse']=train_data['all_word'].apply(len)
# train_data['length_of_sentinse']

In [ ]:
w=[]
for i in all_word:
    w.append(set(i))
train_data['unique_words']=w
train_data['number_of_unique_words']=train_data['unique_words'].apply(len)

In [ ]:
def histogram(length_of_sentinse,number_of_unique_words):
    plt.figure(figsize=(10, 6))
    plt.subplot(2, 1, 1)
    # plt.bar(number_of_unique_words, length_of_sentinse, color='blue', alpha=0.7)
    plt.hist(length_of_sentinse, label='length_of_sentinse',edgecolor='black', linewidth=1.2,bins=30, alpha=0.5)
    plt.hist(number_of_unique_words, label='number_of_unique_words',edgecolor='black', linewidth=1.2,bins=30, alpha=0.5)
    plt.title('histogram')
    plt.xlabel('length_of_sentinse')
    plt.ylabel('number_of_unique_words')
    # plt.tight_layout()
    plt.legend()
    plt.show()

In [ ]:
# لكل النص
histogram(train_data['length_of_sentinse'],train_data['number_of_unique_words'])

In [ ]:
# أمراض الجهاز التنفسي
histogram(train_data[train_data['label']=='أمراض الجهاز التنفسي']['length_of_sentinse'],train_data[train_data['label']=='أمراض الجهاز التنفسي']['number_of_unique_words'])

In [ ]:
# أمراض الدم
histogram(train_data[train_data['label']=='أمراض الدم']['length_of_sentinse'],train_data[train_data['label']=='أمراض الدم']['number_of_unique_words'])

In [ ]:
# أمراض الغدد الصماء
histogram(train_data[train_data['label']=='أمراض الغدد الصماء']['length_of_sentinse'],train_data[train_data['label']=='أمراض الغدد الصماء']['number_of_unique_words'])

In [ ]:
# ارتفاع ضغط الدم
histogram(train_data[train_data['label']=='ارتفاع ضغط الدم']['length_of_sentinse'],train_data[train_data['label']=='ارتفاع ضغط الدم']['number_of_unique_words'])

In [ ]:
# الأورام الخبيثة والحميدة
histogram(train_data[train_data['label']=='الأورام الخبيثة والحميدة']['length_of_sentinse'],train_data[train_data['label']=='الأورام الخبيثة والحميدة']['number_of_unique_words'])

In [ ]:
# جراحة العظام
histogram(train_data[train_data['label']=='جراحة العظام']['length_of_sentinse'],train_data[train_data['label']=='جراحة العظام']['number_of_unique_words'])

In [ ]:
# جراحة عامة
histogram(train_data[train_data['label']=='جراحة عامة']['length_of_sentinse'],train_data[train_data['label']=='جراحة عامة']['number_of_unique_words'])

In [ ]:
# مرض السكري
histogram(train_data[train_data['label']=='مرض السكري']['length_of_sentinse'],train_data[train_data['label']=='مرض السكري']['number_of_unique_words'])

# Question [3]: Text Cleaning and Normalization

## [3.1] 


In [ ]:
# train_data
# clean_tweet = re.sub(r'^RT(\s)+|https?\S+|#|@\S+', '', tweet) ## من أجل أزالت 
#r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
def remove_links(text):
    clean_text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    return clean_text


In [ ]:
# remove_links('https://chat.openai.com/c/dc21a08a-5be6-42e3-a6e4-0c1d26791a46 hi')

## [3.2] 

In [ ]:
def remove_doctors_prefix(text):
    # pattern = r'\bد\.\s*([\u0600-\u06FF]+)\b' # لحتى يشيل ال د قبل الاسم...بالنسبة للأصفار والأرقام هي للغة العربية
    # pattern = r'\bالدكتور\b'
    pattern = r'\bالدكتور(?:ة)?\b'
    result = re.sub(pattern, 'د.', text)
    return result

In [ ]:
texts = train_data[train_data['label']=='أمراض الدم']['answer'].values
for text in texts:
    # print(text)
    print(remove_doctors_prefix(text))
    print("=================================================================")

## [3.3] 

In [ ]:
# def remove_sentences_after_doctor(text):
#     pattern = r'الدكتور\s*[\u0600-\u06FF]+\s*(.*?)(?:[\r\n]{2,}|$)'
#     result = re.sub(pattern, '', text)
#     return result

# text = "كيف الحال\ مرحبًا، الدكتور أحمد.\nكيف حالكم؟\nهذه جملة اختبارية."
# print(remove_sentences_after_doctor(text))



# not done yet

## [3.4] 

In [ ]:
def remove_duplicate_letters_except_alif(text):
    pattern = re.compile(r'(?:(?<!ا)ا(?!ا)|(.)(?:\1)+)', re.UNICODE) # بشلي كلشي أحرف مكررة وخاصة الألف مابكررها اكتر من مرة
    result = pattern.sub(lambda x: x.group(1) if x.group(1) else 'ا', text)
    return result

for text in texts:
    print(remove_duplicate_letters_except_alif(text))

## [3.5]

### [3.5.1] 

In [ ]:
# هاد بيقلب لل 1و2و3و4و....
def convert_numbers_to_arabic_v2(text):
    digit_mapping = {'٠': '0', '١': '1', '٢': '2', '٣': '3', '٤': '4', '٥': '5', '٦': '6', '٧': '7', '٨': '8', '٩': '9'}
    english_text = re.sub(r'[٠-٩]', lambda x: digit_mapping[x.group()], text)

    return english_text


for text in texts:
    print(convert_numbers_to_arabic_v2(text))

In [ ]:
# هاد بيلقلب لل ١,٢,٣,٤
def convert_numbers_to_arabic(text):
    digit_mapping = {'0': '٠', '1': '١', '2': '٢', '3': '٣', '4': '٤', '5': '٥', '6': '٦', '7': '٧', '8': '٨', '9': '٩'}
    arabic_text = re.sub(r'[0-9]', lambda x: digit_mapping[x.group()], text)

    return arabic_text


for text in texts:
    print(convert_numbers_to_arabic(text))

### [3.5.2] 

In [ ]:
def standardize_numbers(text, replacement_symbol='T'):
    digit_pattern = r'[0-9٠-٩]+'
    standardized_text = re.sub(digit_pattern, lambda x: replacement_symbol, text)

    return standardized_text


for text in texts:
    print(standardize_numbers(text))

### [3.5.3] 

In [ ]:
import re

def remove_numbers(text):
    digit_pattern = r'\d+'
    
    text_without_numbers = re.sub(digit_pattern, '', text)

    return text_without_numbers

for text in texts:
    print(remove_numbers(text))

### [3.5.4]

In [ ]:
texts = train_data[train_data['label']=='أمراض الجهاز التنفسي']['answer'].values

def remove_dates_and_times(text):
    date_time_patterns = [
        r'\d{4}-\d{2}-\d{2}\s+\d{1,2}:\d{2}:\d{2}'  # Match dates and times like 2015-01-04 20:08:51
        # r'\d{1,2}/\d{1,2}/\d{2,4}',         # Match dates like 12/31/2022
        # r'\d{1,2}-\d{1,2}-\d{2,4}',         # Match dates like 12-31-2022
        # r'\d{1,2}:\d{2}:\d{2}'               # Match times like 12:34:56
        # r'\d{1,2}\s+(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s*\d{2,4}',  # Match dates like 31 Dec 2022
        # r'\d{1,2}:\d{2}\s*(?:AM|PM|am|pm)?',  # Match times like 12:34 AM
    ]

    for pattern in date_time_patterns:
        text = re.sub(pattern, '', text)

    return text

for text in texts:
    print(remove_dates_and_times(text))

## [3.6] 

### [3.6.1] 

In [ ]:
import string
string.punctuation

In [ ]:
def remove_punctuation(text):
    text=re.sub(r'[!\"#$%&\'()*+,\-./:;<=>?@\[\\\]^_`{|}~،؛؟ـ]',' ',text)
    return text

In [ ]:
text = "مرحباً، هذا نص تجريبي! هل يعمل؟"
clean_text = remove_punctuation(text)
print(clean_text)
for text in texts:
    print(remove_punctuation(text))

### [3.6.2] 

In [ ]:
def keep_arabic_punctuations_numbers(text):
    text= re.sub(r'[^\u0600-\u06FF0-9،؛؟ـ!\"#$%&\'()*+,\-./:;<=>?@\[\\\]^_`{|}~]', ' ', text)
    return text

In [ ]:
text = "مرحباً، هذا نص تجريبي بالعربية! هل يعمل؟ English words 1234"
clean_text = keep_arabic_punctuations_numbers(text)
print(clean_text)
for text in texts:
    print(keep_arabic_punctuations_numbers(text))

## [3.7] 

## [3.8] 

In [ ]:
ar_stemmer=stemmer("arabic")
ar_stemmer.stemWord(u"فسميتموها")

In [ ]:
def stem_arabic_words_farasa(text):
    stemmer = FarasaStemmer()
    text=stemmer.stem(text)
    return text

text = "الكتب في المكتبة متنوعة ومفيدة"
stemmed_text = stem_arabic_words_farasa(text)
print(stemmed_text)

In [ ]:
def stem_arabic_words_tashaphyne(text):
    stemmer = ArabicLightStemmer()
    words = text.split()
    stemmed_words = [stemmer.light_stem(word) for word in words]
    return ' '.join(stemmed_words)

text = "الكتب في المكتبة متنوعة ومفيدة"
stemmed_text = stem_arabic_words_tashaphyne(text)
print(stemmed_text)

In [ ]:
def stem_arabic_words_snowballstemmer(text):
    ar_stemmer = stemmer("arabic")
    words = text.split()
    stemmed_words = [ar_stemmer.stemWord(word) for word in words]
    return ' '.join(stemmed_words)

text = "الكتب في المكتبة متنوعة ومفيدة"
stemmed_text = stem_arabic_words_snowballstemmer(text)
print(stemmed_text)

In [ ]:
# for text in texts:
#     print(stem_arabic_words_snowballstemmer(text))

In [ ]:
# for text in texts:
#     print(stem_arabic_words_tashaphyne(text))

In [ ]:
# for text in texts:
#     print(stem_arabic_words_farasa(text))

## [3.9] 

### [3.9.1] 

In [ ]:
def unify_hamzat(text):
    unified_text = re.sub(r'[ءؤئ]', 'ء', text)
    return unified_text

In [ ]:
text = "أحمد قرأ القرآن بتأنٍ وإتقان"
unified_text = unify_hamzat(text)
print(unified_text)

### [3.9.2] 

In [ ]:
def unify_Alfat(text):
    unified_text = re.sub(r'[أإآ]', 'أ', text)
    return unified_text

In [ ]:
text = "أحمد قرأ القرآن بتأنٍ وإتقان"
unified_text = unify_Alfat(text)
print(unified_text)

### [3.9.3] 

In [ ]:
def remove_tatweel(text):
    text =re.sub( r'ـ', '', text)
    return text

In [ ]:
text = "اللغة العربيــــــة جميلة"
clean_text = remove_tatweel(text)
print(clean_text)

### [3.9.4] 

In [ ]:
def remove_arabic_diacritics(text):
    text=re.sub(r'[\u064B-\u065F]', '', text)
    return text


In [ ]:
for text in texts:
    print(remove_arabic_diacritics(text))

## [3.10] 

In [ ]:
def remove_space(text):
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [ ]:
# text=" sdasd  dasads  das dsa  ads "
# remove_space(text)

## Extra [3.11]

# Question [4]: Prepare Training Data

## [4.1] 


In [ ]:
def preparing_training_data(text):
    text_after_cleaning_links = remove_links(text)
    text_after_removing_doctors_prefix = remove_doctors_prefix(text_after_cleaning_links)
    # text after removing doctors specialization
    text_after_removing_doublication_chars = remove_duplicate_letters_except_alif(text_after_removing_doctors_prefix)
    text_after_removing_date_and_time = remove_dates_and_times(text_after_removing_doublication_chars)
    text_after_removing_numbers = remove_numbers(text_after_removing_date_and_time)
    text_after_removing_non_arabic_words = keep_arabic_punctuations_numbers(text_after_removing_numbers)
    # text after removing stop words
    text_after_unify_hamzat = unify_hamzat(text_after_removing_non_arabic_words)
    text_after_unify_alafat = unify_Alfat(text_after_unify_hamzat)
    text_after_removing_tatwel = remove_tatweel(text_after_unify_alafat)
    text_after_removing_tashkel = remove_arabic_diacritics(text_after_removing_tatwel)
    text_after_removing_spaces_and_keeping_one_space = remove_space(text_after_removing_tashkel)
    # text after removing wrong arabic words and correct them


    return text_after_removing_spaces_and_keeping_one_space
    
    
    
questions = train_data['question'].values
answers = train_data['answer'].values

questions_answers = questions[0:5] + answers[0:5]

questions_answers.tolist()
for i in questions_answers:
    print(preparing_training_data(i))
    
    

## [4.2] 

In [ ]:
all_train_data = questions + answers
all_train_data_After_cleaning = []
all_tokens = []
for line in all_train_data:
    training_data_after_cleaning = preparing_training_data(line)
    all_train_data_After_cleaning.append(training_data_after_cleaning)
    # print(training_data_after_cleaning)



# all_train_data_After_cleaning
# # print(tokens)
for phrase in all_train_data_After_cleaning:
    tokens = nltk.word_tokenize(phrase)
    all_tokens.append(tokens)   


flattened_all_tokens = [item for sublist in all_tokens for item in sublist]
print(len(flattened_all_tokens)) 

In [ ]:
# حساب عدد الكلمات الفريدة :
total_unique_words=len(set(flattened_all_tokens))
print("عدد الكلمات الفريدة:", total_unique_words)

In [ ]:
# نلاحظ ان عدد الكلمات الكلية قد قلت هنا أما عدد الكلمات المميزة قد زاد

## [4.3] 

## [4.4] 

# Question [5]: Baseline

The Comparison Dictionary example

In [ ]:
comparison_table = {}

comparison_table['question_step_number'] = []
comparison_table['model_name'] = []
comparison_table['features'] = []
comparison_table['model_parameters'] = []
comparison_table['preprocessing_methods'] = []
comparison_table['accuracy'] = []
comparison_table['balance_accuracy'] = []

Filling the dictionary example

In [ ]:
comparison_table['question_step_number'].append("3.5")
comparison_table['model_name'].append('logistic_regression')
comparison_table['features'].append('bag of words')
comparison_table['model_parameters'].append('default')
comparison_table['preprocessing_methods'].append("remove links")
comparison_table['accuracy'].append(accuracy)
comparison_table['balance_accuracy'].append(balance_accuracy_value)

## [5.1]

## [5.2]

## [5.3]

## [5.4]

## [5.5]

## [5.6]

# Question [6]: Semantic Representation

## [6.1]

## [6.2]

## [6.3]

## [6.4]

## [6.5]

# Question [7]: Deep Neural Network

## [7.1]

### [7.1.1]

### [7.1.2]

## [7.2]

### [7.2.1]

### [7.2.2]

### [7.2.3]

### [7.2.4]

## [7.3]

## Extra [7.4]

# Final Results

In [ ]:
df = pd.DataFrame(comparison_table)
df

In [ ]:
df.to_csv("[your_name].csv", index=False)